In [ ]:
# instalação dos pacotes necessários na versão mais nova
%pip install -U scikit-learn pandas numpy seaborn

In [208]:
# importando bibliotecas necessárias
import pandas as pd
import numpy as np

# graphs
import matplotlib.pyplot as plt
import seaborn as sns

# separação de dados
from sklearn.model_selection import train_test_split

# padronização dos dados
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# construção de filtro para ColumnTransformer
from sklearn.compose import make_column_selector

# criação de pipeline para tratamento do conjunto de dados
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# modelo
from sklearn.linear_model import LassoCV, RidgeCV

In [176]:
# agora uma função que vai printar bonitinho as métricas
def print_metrics(model: Pipeline, model_name: str, X_test: np.ndarray, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the accuracy, precision and recall scores of the model
      
      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA
      
      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.
  
  """
  #imports
  from sklearn.metrics import mean_absolute_error, mean_squared_error

  return (f'''

Mean Absolute Error (MAE) do Modelo {model_name} sem PCA: {mean_absolute_error(y_test, y_pred):.3f}

Mean Squared Error (MSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=True):.4f}

Root Mean Squared Error (RMSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=False):.4f}

R-squared do Modelo {model_name} sem PCA: {model.score(X_test, y_test):.3f}
''')

In [131]:
from warnings import simplefilter
#from sklearn.exceptions import UserWarning

simplefilter("ignore", category=UserWarning)

## 1. Análise exploratória de dados (1.0 ponto)

Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.


In [178]:
# carregando dataset de teste

dataframe = pd.read_csv('dado/train.csv')
dataframe_test = pd.read_csv('dado/test.csv')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [202]:
# Separando dados de treinamento do modelo
X = dataframe.drop(['SalePrice'], axis=1)
y = dataframe.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

info on onehotencoder:

It is also possible to encode each column into n_categories - 1 columns instead of n_categories columns by using the drop parameter. This parameter allows the user to specify a category for each feature to be dropped. This is useful to avoid co-linearity in the input matrix in some classifiers. Such functionality is useful, for example, when using non-regularized regression (LinearRegression), since co-linearity would cause the covariance matrix to be non-invertible:

## 2. Levantamento de hipóteses (1.0 ponto)

Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.



## 3. Apresentação das ideias obtidas (1.0 ponto)

Apresente com gráficos as suas observações e a descreva cada gráfico.


## 4. Preparação dos dados (1.0 ponto)

Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados



#### Steps
- Outliers
- Encoding
- Impute Missing Values
- Scaler
- PCA
- Model
- Cross-Val

In [194]:
# filtros para o pipeline

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV sem pca

numeric_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
        ]
)

categorical_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer, col_num),
    ("cat", categorical_transformer, col_tex)
  ]
)

modelLassoCV = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f86126fdb10>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f8610a834f0>)])),
                ('lassocv', LassoCV(cv=5))])

In [204]:
# fit e predict do modelo
modelLassoCV.fit(X_train, y_train)
y_pred = modelLassoCV.predict(X_test)

In [200]:
# obtendo as features selecionadas pelo LassoCV

preprocessor_fitted = modelLassoCV['preprocessor']

feature_names = (
  list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
  list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
)

feature_names = np.array(feature_names)

kept = ~np.isclose(modelLassoCV['lassocv'].coef_, 0)

print(f'''
Número de Features Original: {len(feature_names)}
Número de Features selecionadas pelo LassoCV: {len(feature_names[kept])}
Features Selecionadas:
{feature_names[kept]}
''')


Número de Features Original: 300
Número de Features selecionadas pelo LassoCV: 90
Features Selecionadas:
['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'TotalBsmtSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath'
 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr'
 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars' 'GarageArea'
 'WoodDeckSF' 'OpenPorchSF' '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal'
 'MoSold' 'x0_RM' 'x3_IR2' 'x4_Bnk' 'x4_HLS' 'x6_CulDSac' 'x7_Mod'
 'x8_BrkSide' 'x8_Crawfor' 'x8_Edwards' 'x8_Gilbert' 'x8_Mitchel'
 'x8_NAmes' 'x8_NWAmes' 'x8_NoRidge' 'x8_NridgHt' 'x8_Somerst'
 'x8_StoneBr' 'x9_Norm' 'x10_PosN' 'x11_1Fam' 'x11_2fmCon' 'x12_1Story'
 'x12_2Story' 'x13_Gable' 'x14_ClyTile' 'x15_BrkFace' 'x15_HdBoard'
 'x16_ImStucc' 'x16_Plywood' 'x17_BrkFace' 'x17_None' 'x18_Ex' 'x18_TA'
 'x20_BrkTil' 'x20_PConc' 'x21_Ex' 'x22_TA' 'x23_Gd' 'x23_No' 'x24_G

In [199]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV sem PCA: 17523.065

Mean Squared Error (MSE) do Modelo Pipeline LassoCV sem PCA: 866285046.5730

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV sem PCA: 29432.7207

R-squared do Modelo Pipeline LassoCV sem PCA: 0.887



## 5. Criação e seleção de características (1.0 ponto)

Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?


Aqui eu faço pipeline com pca

In [192]:
# filtros para o pipeline com pca

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelLassoCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f86126fe350>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f86126fe230>)])),
                ('lassocv', LassoCV(cv=5))])

In [205]:
# fit e predict do modelo LassoCV com pca
modelLassoCV_pca.fit(X_train, y_train)
y_pred = modelLassoCV_pca.predict(X_test)

In [206]:
# obtendo as features selecionadas pelo LassoCV com PCA

preprocessor_fitted = modelLassoCV_pca['preprocessor']

feature_names = (
  list(preprocessor_fitted.named_transformers_['num']['pca'].get_feature_names_out()) +
  list(preprocessor_fitted.named_transformers_['cat']['pca'].get_feature_names_out())
)

feature_names = np.array(feature_names)

kept = ~np.isclose(modelLassoCV_pca['lassocv'].coef_, 0)

print(f'''
Número de Features Original: {len(feature_names)}
Número de Features selecionadas pelo LassoCV: {len(feature_names[kept])}
Features Selecionadas:
{feature_names[kept]}
''')


Número de Features Original: 250
Número de Features selecionadas pelo LassoCV: 108
Features Selecionadas:
['pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6' 'pca7' 'pca8' 'pca9'
 'pca10' 'pca11' 'pca14' 'pca15' 'pca16' 'pca17' 'pca18' 'pca19' 'pca20'
 'pca22' 'pca23' 'pca24' 'pca25' 'pca26' 'pca27' 'pca28' 'pca29' 'pca31'
 'pca32' 'pca33' 'pca34' 'pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6'
 'pca7' 'pca8' 'pca9' 'pca10' 'pca12' 'pca13' 'pca14' 'pca15' 'pca16'
 'pca17' 'pca18' 'pca19' 'pca20' 'pca22' 'pca23' 'pca24' 'pca25' 'pca26'
 'pca27' 'pca29' 'pca32' 'pca33' 'pca34' 'pca35' 'pca36' 'pca39' 'pca40'
 'pca41' 'pca42' 'pca43' 'pca44' 'pca45' 'pca47' 'pca48' 'pca51' 'pca52'
 'pca53' 'pca55' 'pca56' 'pca57' 'pca59' 'pca60' 'pca61' 'pca62' 'pca64'
 'pca67' 'pca68' 'pca69' 'pca71' 'pca75' 'pca76' 'pca78' 'pca82' 'pca85'
 'pca86' 'pca88' 'pca89' 'pca94' 'pca96' 'pca98' 'pca99' 'pca102' 'pca104'
 'pca106' 'pca107' 'pca118' 'pca131' 'pca180' 'pca182' 'pca184']



In [207]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV sem PCA: 18956.904

Mean Squared Error (MSE) do Modelo Pipeline LassoCV sem PCA: 978705610.2881

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV sem PCA: 31284.2710

R-squared do Modelo Pipeline LassoCV sem PCA: 0.872



### Testando o Pipeline com Ridge e pca

In [ ]:
# filtros para o pipeline com ridge e pca

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelRidgeCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", RidgeCV(cv=5))
  ]
)

modelRidgeCV_pca

## 6. Decisão do modelo (1.0 ponto)

Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?



aqui eu escolho o pipeline com melhor resultado

## 7. Modelagem (1.0 ponto)

Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.



aqui eu fito e printo as métricas de novo
se pá uso um  GridSearchCV

## 8. Histórico (1.0 pontos)

Preencha a tabela abaixo com cada teste realizado com seu modelo. Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.


tabelinha de métricas
#testar ridge

## 9. Resultado (2.0 pontos)

Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.


escolher